# NBA Klout working notebook
In this notebook we:
- have scrapers for Basketball Reference, HoopsHype, Instagram.
- use APIs for NBA Stats and twitter
- create a Postgres server
- create a table containing all the stats
- Build the tables for the live demo


## here we get the players from the roster

In [11]:
#get teams
from nba_api.stats.endpoints import leaguestandings
nba = '00'
season21 = '2021-22'
regseason = 'Regular Season'
teams = leaguestandings.LeagueStandings(league_id=nba, season=season21, season_type=regseason).standings.get_data_frame()
teams.to_csv('teams_standings.csv', index=False)
teams.head()

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22021,1610612756,Phoenix,Suns,West,25-7,1,,Pacific,...,None,None,None,None,None,2-3,16-0,9-5,39-9,None
1,00,22021,1610612748,Miami,Heat,East,20-10,1,,Southeast,...,None,None,None,None,None,5-1,8-7,10-5,32-18,None
2,00,22021,1610612741,Chicago,Bulls,East,20-11,2,,Central,...,None,None,None,None,None,5-1,9-7,9-2,30-18,None
3,00,22021,1610612744,Golden State,Warriors,West,20-8,2,,Pacific,...,None,None,None,None,None,5-1,13-2,9-4,37-13,None
4,00,22021,1610612755,Philadelphia,76ers,East,17-12,3,,Atlantic,...,None,None,None,None,None,4-2,7-8,8-6,30-19,None


In [13]:
#get roster from teamid
#    team['TeamID']
from nba_api.stats.endpoints import commonteamroster
roster = commonteamroster.CommonTeamRoster(season=season21, team_id=1610612756).common_team_roster.get_data_frame()
roster.head()

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID
0,1610612756,2021,00,JaVale McGee,JaVale,javale-mcgee,00,C-F,7-0,270,"JAN 19, 1988",34.0,13,Nevada-Reno,201580
1,1610612756,2021,00,Devin Booker,Devin,devin-booker,1,G,6-5,206,"OCT 30, 1996",25.0,6,Kentucky,1626164
2,1610612756,2021,00,Elfrid Payton,Elfrid,elfrid-payton,2,G,6-3,195,"FEB 22, 1994",27.0,7,Louisana-Lafayette,203901
3,1610612756,2021,00,Chris Paul,Chris,chris-paul,3,G,6-0,175,"MAY 06, 1985",36.0,16,Wake Forest,101108
4,1610612756,2021,00,Frank Kaminsky,Frank,frank-kaminsky,8,F-C,7-0,240,"APR 04, 1993",28.0,6,Wisconsin,1626163


In [55]:
import pandas as pd
import time
#get players from team rosters
roster = commonteamroster.CommonTeamRoster(season=season21, team_id=1610612756).common_team_roster.get_data_frame()
players= pd.DataFrame(columns=roster.columns)
team_list = teams['TeamID']
time.sleep(30)
for team in team_list:
    roster = commonteamroster.CommonTeamRoster(season=season21, team_id=int(team)).common_team_roster.get_data_frame()
    players = players.append(roster)
    time.sleep(30)

In [57]:
#save players list
players.to_csv('active_players.csv', index=False)
players.head()

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID
0,1610612756,2021,00,JaVale McGee,JaVale,javale-mcgee,00,C-F,7-0,270,"JAN 19, 1988",34.0,13,Nevada-Reno,201580
1,1610612756,2021,00,Devin Booker,Devin,devin-booker,1,G,6-5,206,"OCT 30, 1996",25.0,6,Kentucky,1626164
2,1610612756,2021,00,Elfrid Payton,Elfrid,elfrid-payton,2,G,6-3,195,"FEB 22, 1994",27.0,7,Louisana-Lafayette,203901
3,1610612756,2021,00,Chris Paul,Chris,chris-paul,3,G,6-0,175,"MAY 06, 1985",36.0,16,Wake Forest,101108
4,1610612756,2021,00,Frank Kaminsky,Frank,frank-kaminsky,8,F-C,7-0,240,"APR 04, 1993",28.0,6,Wisconsin,1626163


## Create stats table

In [21]:

import nba_api.stats.endpoints
#import nba_api.stats.parameters

players_games = cumestatsplayergames.CumeStatsPlayerGames(
    league_id = 'nba' ,
    player_id = 203500,
    season = '2021-22',
    season_type_all_star = 'regular')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# CONNECT TO SQL SERVER

### Using Postgres

In [38]:
import psycopg2

conn = psycopg2.connect(
    database=dbname,
    user=username,
    password=passwd,
    host=hostname,
    port=portnum
)
cur = conn.cursor()

In [42]:
query = 'SELECT * FROM information_schema.tables;'
cur.execute(query)
cur.fetchall()

[('Players',
  'public',
  'Salary',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_type',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_foreign_table',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_roles',
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_settings',
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_prepared_statements',
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_stat_progress_vacuum',
  'VIEW',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NO',
  None),
 ('Players',
  'pg_catalog',
  'pg_subscription',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO

In [41]:
query = 'select * from public."Salary";'
cur.execute(query)
cur.fetchall()

[(0,
  'Stephen Curry',
  'GSW',
  '$45,780,966',
  '$48,070,014',
  '$51,915,615',
  '$55,761,216',
  '$59,606,817',
  '',
  'Bird Rights',
  '$261,134,628'),
 (1,
  'John Wall',
  'HOU',
  '$44,310,840',
  '$47,366,760',
  '',
  '',
  '',
  '',
  'Bird Rights',
  '$44,310,840'),
 (2,
  'Russell Westbrook',
  'LAL',
  '$44,211,146',
  '$47,063,478',
  '',
  '',
  '',
  '',
  'Bird Rights',
  '$44,211,146'),
 (3,
  'James Harden',
  'BRK',
  '$43,848,000',
  '$46,872,000',
  '',
  '',
  '',
  '',
  'Bird Rights',
  '$43,848,000'),
 (4,
  'LeBron James',
  'LAL',
  '$41,180,544',
  '$44,474,988',
  '',
  '',
  '',
  '',
  'Bird',
  '$85,655,532'),
 (5,
  'Kevin Durant',
  'BRK',
  '$40,918,900',
  '$42,969,845',
  '$42,969,845',
  '$49,856,021',
  '$53,282,609',
  '',
  'Sign and Trade',
  '$229,997,220'),
 (6,
  'Giannis Antetokounmpo',
  'MIL',
  '$39,344,970',
  '$42,492,568',
  '$45,640,165',
  '$48,787,763',
  '$51,935,360',
  '',
  'Bird',
  '$176,265,466'),
 (7,
  'Paul George',


### Using sqlalchemy

In [37]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import inspect
import psycopg2
import pandas as pd


engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(username,passwd,hostname,portnum,dbname))
session = scoped_session(sessionmaker(bind=engine))
inspector = inspect(engine)

In [40]:
inspector.get_columns("Salary", "public")
r = engine.execute('select * from public."Salary";')  
query = "select * from public."Salary";"

In [44]:
conn = engine.connect()
query = 'select * from public."Salary";'
df = pd.read_sql(query,conn)
df

,index,Player,Tm,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27,Signed Using,Guaranteed
0,0,Stephen Curry,GSW,"$45,780,966","$48,070,014","$51,915,615","$55,761,216","$59,606,817",,Bird Rights,"$261,134,628"
1,1,John Wall,HOU,"$44,310,840","$47,366,760",,,,,Bird Rights,"$44,310,840"
2,2,Russell Westbrook,LAL,"$44,211,146","$47,063,478",,,,,Bird Rights,"$44,211,146"
3,3,James Harden,BRK,"$43,848,000","$46,872,000",,,,,Bird Rights,"$43,848,000"
4,4,LeBron James,LAL,"$41,180,544","$44,474,988",,,,,Bird,"$85,655,532"
...,...,...,...,...,...,...,...,...,...,...,...
577,577,Paris Bass,PHO,"$53,176",,,,,,Minimum Salary,"$106,352"
578,578,Dakota Mathias,MEM,"$53,176",,,,,,,"$106,352"
579,579,Dakota Mathias,MEM,"$53,176",,,,,,Minimum Salary,"$106,352"
580,580,Malcolm Hill,CHI,,,,,,,,"$53,176"


In [36]:
session.close()

In [5]:
stats.to_sql('Salary', engine)

NameError: name 'stats' is not defined

## Close connections

In [140]:
engine.dispose()
conn.close()

# GET SALARY

## hoopshype salaries

In [175]:
import pandas as pd
from selenium.webdriver import Chrome 
from selenium.webdriver.common.by import By
driver = Chrome()
df = pd.DataFrame(columns=['Player','Salary','Year'])
url = 'https://hoopshype.com/salaries/players/'
driver.get(url)
players = driver.find_elements(By.XPATH,'//td[@class="name"]')
salaries = driver.find_elements(By.XPATH,'//td[@data-value]') 
players_list = []
for p in range(len(players)):
    players_list.append(players[p].text)
salaries_list = []
for s in range(len(salaries)):
    if salaries[s].text == '':
        salaries_list.append('$0')
    else:
        salaries_list.append(salaries[s].text)   
driver.quit()

In [250]:
data=[]
def dollar_to_int(s):
    st = s.replace('$','')
    return st.replace(',','')
def clear_name(s):
    st = s.replace('\'','')
    return st.replace('.','')

csalaries_list = list(map(dollar_to_int,salaries_list))
players = players_list[1:]
for i in range(len(players)):
    tup = (clear_name(players[i]), csalaries_list[i*6],
           csalaries_list[(i*6)+1],csalaries_list[(i*6)+2],
           csalaries_list[(i*6)+3],csalaries_list[(i*6)+4],
           csalaries_list[(i*6)+5])
    data.append(tup)
hh_salary = pd.DataFrame(data, columns=['Name','2021-22','2022-23','2023-24','2024-25','2025-26','2026-27'])
hh_salary.to_csv('hhsalary.csv', index=False)
hh_salary

,Name,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27
0,Stephen Curry,45780966,48070014,51915615,55761217,0,0
1,James Harden,44310840,47366760,0,0,0,0
2,John Wall,44310840,47366760,0,0,0,0
3,Russell Westbrook,44211146,47063478,0,0,0,0
4,LeBron James,41180544,44474988,0,0,0,0
...,...,...,...,...,...,...,...
638,Jaime Echenique,53176,0,0,0,0,0
639,Jordan Goodwin,53176,0,0,0,0,0
640,Jordan Schakel,53176,0,0,0,0,0
641,Craig Sword,53176,0,0,0,0,0


# SEARCH FOR INSTAGRAM HANDLE

In [ ]:
#importing the libraries we will need
import pandas as pd
import numpy as np
import urllib
from fake_useragent import UserAgent
import requests
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [81]:
keyword= "instagram daishen nix"
html_keyword= urllib.parse.quote_plus(keyword)
print(html_keyword)

instagram+daishen+nix


In [82]:
number_of_result=10
google_url = "https://www.google.com/search?q=" + html_keyword + "&num=" + str(number_of_result)
print(google_url)

https://www.google.com/search?q=instagram+daishen+nix&num=10


In [83]:

response = requests.get(google_url)
soup = BeautifulSoup(response.text, "html.parser")

In [89]:
result = soup.find_all('div', attrs = {'class': 'ZINbbc'})
results=[re.search('\/url\?q\=(.*)\&sa',str(i.find('a', href = True)['href'])) for i in result if "url" in str(i)]
#this is because in rare cases we can't get the urls
links=[i.group(1) for i in results if i != None]
links

['https://www.instagram.com/djdaishen/%3Fhl%3Den',
 'https://www.instagram.com/daishen_nix/%3Fhl%3Den',
 'https://www.instagram.com/p/q3OXvaNg3v/%3Fhl%3Den',
 'https://twitter.com/djfromveg%3Fref_src%3Dtwsrc%255Egoogle%257Ctwcamp%255Eserp%257Ctwgr%255Eauthor',
 'https://www.instagram.com/p/B_iF3C3pdFq/%3Fhl%3Den',
 'https://www.instagram.com/p/CYA5zIeukwX/',
 'https://www.instagram.com/p/B7rSsOjlrVc/',
 'https://www.instagram.com/p/CEhmzeylDFT/',
 'https://www.instagram.com/daishenig/',
 'https://www.basketball-reference.com/players/n/nixda01.html']

In [100]:
test = re.findall(r'^(.*)/%', links[0])[0]
test

'https://www.instagram.com/djdaishen'

In [101]:
#links[0].split("%")[0].replace("https://www.instagram.com/","")[:-1]
test.replace("https://www.instagram.com/","")

'djdaishen'

In [ ]:
#search for players instagram handle
#input: player as string
#output: handle as string
def handle_search(player, social):
    #importing the libraries we will need
    import pandas as pd
    import numpy as np
    import urllib
    from fake_useragent import UserAgent
    import requests
    import re
    from urllib.request import Request, urlopen
    from bs4 import BeautifulSoup
    
    keyword= social + " " + player
    html_keyword= urllib.parse.quote_plus(keyword)
    number_of_result=1
    google_url = "https://www.google.com/search?q=" + html_keyword + "&num=" + str(number_of_result)
    response = requests.get(google_url)
    soup = BeautifulSoup(response.text, "html.parser")
    result = soup.find_all('div', attrs = {'class': 'ZINbbc'})
    results=[re.search('\/url\?q\=(.*)\&sa',str(i.find('a', href = True)['href'])) for i in result if "url" in str(i)]
    #this is because in rare cases we can't get the urls
    links=[i.group(1) for i in results if i != None]
    if link[0][0].contains(social+".com"):
        test = re.findall(r'^(.*)/%', links[0])[0]
        return test
    else:
        return "None"
        
    

# SEARCH FOR TWITTER HANDLE


In [102]:
keyword= "twitter daishen nix"
html_keyword= urllib.parse.quote_plus(keyword)
print(html_keyword)
number_of_result=10
google_url = "https://www.google.com/search?q=" + html_keyword + "&num=" + str(number_of_result)
print(google_url)
response = requests.get(google_url)
soup = BeautifulSoup(response.text, "html.parser")
result = soup.find_all('div', attrs = {'class': 'ZINbbc'})
results=[re.search('\/url\?q\=(.*)\&sa',str(i.find('a', href = True)['href'])) for i in result if "url" in str(i)]
#this is because in rare cases we can't get the urls
links=[i.group(1) for i in results if i != None]
links

twitter+daishen+nix
https://www.google.com/search?q=twitter+daishen+nix&num=10


['https://twitter.com/djfromveg%3Fref_src%3Dtwsrc%255Egoogle%257Ctwcamp%255Eserp%257Ctwgr%255Eauthor',
 'https://twitter.com/daishenn',
 'https://twitter.com/teamdaishennix',
 'https://twitter.com/qakashowtyme%3Flang%3Den',
 'https://twitter.com/shamscharania/status/1255170867916611592',
 'https://mobile.twitter.com/djfromveg/with_replies',
 'https://twitter.com/nbagleague/status/1468405492938649603',
 'https://twitter.com/ballislife/status/1421243708498800642',
 'https://twitter.com/ShamsCharania/status/1449066424362840065',
 'https://mobile.twitter.com/draftexpress/status/1255178997518012418']

In [80]:
links[0].split("%")[0].replace("https://twitter.com/","")

'djfromveg'

In [104]:
test = re.findall(r'^(.*)/', links[0])[0]
test

'https://twitter.com'

# BASKETBALL REFERENCE SCRAPING API

## Get suffix of html for basketball player

In [165]:
from requests import get
from bs4 import BeautifulSoup
import requests
import re
import unicodedata, unidecode

def create_last_name_part_of_suffix(potential_last_names):
    last_names = ''.join(potential_last_names)
    if len(last_names) <= 5:
        return last_names[:].lower()
    else:
        return last_names[:5].lower()


def get_player_suffix(aname):
    outliers ={
        'Metta World Peace': '/players/a/artesro01.html',
        'PJ Tucker':'/players/t/tuckepj01.html',
        'Cedi Osman':'/players/o/osmande01.html',
        'Frank Ntilikina':'/players/n/ntilila01.html',
        'Maxi Kleber':'/players/k/klebima01.html',
        'DeAnthony Melton': '/players/m/meltode01.html',
        'David Duke Jr': '/players/d/dukeda01.html',
        'DayRon Sharpe': '/players/s/sharpda01.html',
        'DeAndre Bembry': '/players/b/bembrde01.html',
        'Enes Freedom': '/players/k/kanteen01.html',
        'DeAndre Hunter': '/players/h/huntede01.html',
        'Clint Capela': '/players/c/capelca01.html',
        'Kevin Knox II': '/players/k/knoxke01.html',
        'Didi Louzada': '/players/l/louzama01.html',
        'Devonte Graham': '/players/g/grahade01.html',
        'DeAaron Fox': '/players/f/foxde01.html',
        'Jahmius Ramsey': '/players/r/ramseja01.html',
        'JaeSean Tate': '/players/t/tateja01.html',
        'PJ Washington': '/players/w/washipj01.html',
        'DAngelo Russell': '/players/r/russeda01.html',
        'TJ Warren': '/players/w/warretj01.html',
        'TJ McConnell': '/players//m/mccontj01.html',
        'RJ Hampton': '/players/h/hamptrj01.html',
        'ETwaun Moore': '/players/m/mooreet01.html',
        'DJ Augustin': '/players/a/augusdj01.html'
    }
    if aname.find('.') != -1:
            name = aname.replace('.','')
    elif aname.find('\'') != -1:
            name = aname.replace('\'','')
    else:
        name = aname
    if name in outliers:
        return outliers[name]     
    normalized_name = unidecode.unidecode(unicodedata.normalize('NFD', name).encode('ascii', 'ignore').decode("utf-8"))
    if normalized_name == 'Metta World Peace' :
        suffix = '/players/a/artesro01.html'
    else:
        if normalized_name.find('.') != -1:
            normalized_name = normalized_name.replace('.','')
        
        split_normalized_name = normalized_name.split(' ')
        if len(split_normalized_name) < 2:
            return None
        initial = normalized_name.split(' ')[1][0].lower()
        
        all_names = name.split(' ')
        first_name_part = unidecode.unidecode(all_names[0][:2].lower())
        first_name = all_names[0]
        other_names = all_names[1:]
        other_names_search = other_names
        last_name_part = create_last_name_part_of_suffix(other_names)
        suffix = '/players/'+initial+'/'+last_name_part+first_name_part+'01.html'
    player_r = get(f'https://www.basketball-reference.com{suffix}')
    while player_r.status_code == 404:
        other_names_search.pop(0)
        last_name_part = create_last_name_part_of_suffix(other_names_search)
        try: 
            initial = last_name_part[0].lower()
        except:
            print(name)
            return None
        suffix = '/players/'+initial+'/'+last_name_part+first_name_part+'01.html'
        player_r = get(f'https://www.basketball-reference.com{suffix}')
    while player_r.status_code==200:
        player_soup = BeautifulSoup(player_r.content, 'html.parser')
        h1 = player_soup.find('h1', attrs={'itemprop': 'name'})
        if h1:
            page_name = h1.find('span').text
            """
                Test if the URL we constructed matches the 
                name of the player on that page; if it does,
                return suffix, if not add 1 to the numbering
                and recheck.
            """
            if ((unidecode.unidecode(page_name)).lower() == normalized_name.lower()):
                return suffix
            else:
                page_names = unidecode.unidecode(page_name).lower().split(' ')
                page_first_name = page_names[0]
                try:
                    if first_name.lower() == page_first_name.lower():
                        return suffix
                # if players have same first two letters of last name then just
                # increment suffix
                    elif first_name.lower()[:2] == page_first_name.lower()[:2]:
                        player_number = int(''.join(c for c in suffix if c.isdigit())) + 1
                        if player_number < 10:
                            player_number = f"0{str(player_number)}"
                        suffix = f"/players/{initial}/{last_name_part}{first_name_part}{player_number}.html"
                    else:
                        other_names_search.pop(0)
                        last_name_part = create_last_name_part_of_suffix(other_names_search)
                        initial = last_name_part[0].lower()
                        suffix = '/players/'+initial+'/'+last_name_part+first_name_part+'01.html'
                except:
                    return None

                player_r = get(f'https://www.basketball-reference.com{suffix}')
    print(name)
    return None

suffix = get_player_suffix("Royce O'Neale")

br_url = 'https://www.basketball-reference.com' + suffix

In [166]:
players = pd.read_csv('active_players.csv')
player_br_url = pd.DataFrame(columns = ['playerid','name','br_url'])
for index, player in players.iterrows():
    name = player['PLAYER']
    br_url = 'https://www.basketball-reference.com' + str(get_player_suffix(name) or "")
    player_br_url.loc[player_br_url.shape[0]] = [player['PLAYER_ID'], name, br_url ]
    
    

In [167]:
player_br_url.to_csv('players_br_urls', index=False)
player_br_url.head()

,playerid,name,br_url
0,201580,JaVale McGee,https://www.basketball-reference.com/players/m...
1,1626164,Devin Booker,https://www.basketball-reference.com/players/b...
2,203901,Elfrid Payton,https://www.basketball-reference.com/players/p...
3,101108,Chris Paul,https://www.basketball-reference.com/players/p...
4,1626163,Frank Kaminsky,https://www.basketball-reference.com/players/k...


## Retrieve Social Handles

In [153]:
from requests import get
from bs4 import BeautifulSoup
import requests
import re
### input: br_url of player
### output: twitter url, 
def get_br_socials(br_url):
    socials = [None]*4
    response = requests.get(br_url)
    soup = BeautifulSoup(response.text, "html.parser")
    try: 
        result =  soup.find("div", {"id": "info"}).find_all('a', href=True)
    except:
        print(br_url)
        return socials

    #get only instagram and twitter links
    results = [ i for i in result if ("twitter.com" in str(i) or ("instagram.com" in str(i))) and i.text != '' ]
    for i in results:
        if "twitter.com" in str(i):
            socials[0] = i['href']
            socials[1] = i.text
        if "instagram.com" in str(i):
            socials[2] = i['href']
            socials[3] = i.text
    return socials

get_br_socials(player_br_url[player_br_url['name'] == 'Devin Booker'].br_url[1])
#player_br_url[player_br_url['name'] == 'Devin Booker'].br_url[1]
    

    

['https://twitter.com/DevinBook',
 'DevinBook',
 'https://instagram.com/dbook',
 'dbook']

In [168]:
for index, player in player_br_url.iterrows():
    if player['br_url'] == 'https://www.basketball-reference.com': 
        print(player['name'])

In [169]:
players_socials = pd.DataFrame(columns = ['playerid','name','twit_url','twit_handle','insta_url','insta_handle'])
for index, player in player_br_url.iterrows():
    result = [player['playerid'], player['name']]
    social = get_br_socials(player['br_url'])
    result.extend(social)
    players_socials.loc[players_socials.shape[0]] = result
players_socials.head(10)

,playerid,name,twit_url,twit_handle,insta_url,insta_handle
0,201580,JaVale McGee,https://twitter.com/JaValeMcGee,JaValeMcGee,https://instagram.com/javalemcgee,javalemcgee
1,1626164,Devin Booker,https://twitter.com/DevinBook,DevinBook,https://instagram.com/dbook,dbook
2,203901,Elfrid Payton,https://twitter.com/elfrid,elfrid,https://instagram.com/elfrid,elfrid
3,101108,Chris Paul,https://twitter.com/CP3,CP3,https://instagram.com/cp3,cp3
4,1626163,Frank Kaminsky,https://twitter.com/FSKPart3,FSKPart3,https://instagram.com/fskpart3,fskpart3
5,1630188,Jalen Smith,https://twitter.com/thejalen_smith,thejalen_smith,https://instagram.com/thejalen_smith,thejalen_smith
6,1627846,Abdel Nader,https://twitter.com/AbdelNader2,AbdelNader2,https://instagram.com/dulenader2,dulenader2
7,1630688,Ish Wainright,https://twitter.com/Wainright24,Wainright24,https://instagram.com/mailman24,mailman24
8,1629013,Landry Shamet,https://twitter.com/landryshamet,landryshamet,https://instagram.com/landryshamet,landryshamet
9,1626166,Cameron Payne,https://twitter.com/campayne,campayne,https://instagram.com/smoothcp1,smoothcp1


In [170]:
players_socials.to_csv('players_socials.csv', index=False)

In [371]:
#correction to mike muscala
import numpy as np
players_socials = pd.read_csv('players_socials.csv')
index = players_socials.index[players_socials['name']=='Mike Muscala'][0]
players_socials['twit_url'] = players_socials['twit_url'].replace(['https://twitter.com/mikemuscala'],np.nan)
players_socials['twit_handle'] = players_socials['twit_handle'].replace(['mikemuscala'],np.nan)
players_socials.loc[index]['twit_handle'] = np.nan
players_socials.to_csv('players_socials.csv', index=False)

# BUILD PLAYERS SOCIALS STATS

## Get twitter stats

In [45]:
import tweepy

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAC%2FjXwEAAAAAUnmfELGGAifch1OktXC%2ByLE0DFs%3D3HaReF7sd1RWcZDuWVDLrZ5I0IMKp66apKOY4cbg1z1p0Y9NEU'
client = tweepy.Client(bearer_token)
''
id = '24838521'

user = client.get_user(username='JaValeMcGee', user_fields=["public_metrics"]).data
user.public_metrics['followers_count']
user.id

24838521

In [105]:
#create blocks of 100
players_socials=pd.read_csv('players_socials_stats.csv')
twit_blocks=[]
block = []
players_socials_stats = players_socials
for index, player in players_socials_stats.iterrows():
    if pd.isna(player['twit_handle']) == False :
        block.append(player['twit_handle'])
        if len(block) > 99:
            twit_blocks.append(block)
            block = []
if len(block) > 0:
    twit_blocks.append(block)

In [106]:
twitter_info = []
for block in twit_blocks:
    info = client.get_users(usernames=block, user_fields=["public_metrics"]).data
    twitter_info.extend(info)
    time.sleep(1)

In [107]:
#change twitter info into dictionary
twitter_info_dict = {}
for info in twitter_info:
    twitter_info_dict[info.username] = {'twit_id' : int(info.id),
                                        'twit_followers_count' : int(info.public_metrics['followers_count']),
                                        'twit_tweet_count' : int(info.public_metrics['tweet_count'])
                                       }

In [108]:
#add info
import numpy as np
id_column = []
followers_column = []
tweet_column = []
for index, player in players_socials_stats.iterrows():
    if pd.isna(player['twit_handle']) == False:
        if player['twit_handle'] in twitter_info_dict:
            info = twitter_info_dict[player['twit_handle']]
            id_column.append(int(info['twit_id']))
            followers_column.append(int(info['twit_followers_count']))
            tweet_column.append(int(info['twit_tweet_count']))
        ##clear incorrect socials
        else:
            players_socials['twit_url'] = players_socials['twit_url'].replace(player['twit_url'],np.nan)
            players_socials['twit_handle'] = players_socials['twit_handle'].replace(player['twit_handle'],np.nan)
            id_column.append(0)
            followers_column.append(0)
            tweet_column.append(0)
    else:
        id_column.append(0)
        followers_column.append(0)
        tweet_column.append(0)
players_socials_stats['twit_id'] = id_column
players_socials_stats['twit_followers_count'] = followers_column
players_socials_stats['twit_tweet_count'] = tweet_column

In [109]:
players_socials_stats.head()

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count
0,201580,JaVale McGee,https://twitter.com/JaValeMcGee,JaValeMcGee,https://instagram.com/javalemcgee,javalemcgee,24838521,365149,12454
1,1626164,Devin Booker,https://twitter.com/DevinBook,DevinBook,https://instagram.com/dbook,dbook,237073728,1089895,9161
2,203901,Elfrid Payton,https://twitter.com/elfrid,elfrid,https://instagram.com/elfrid,elfrid,253527338,66291,10892
3,101108,Chris Paul,https://twitter.com/CP3,CP3,https://instagram.com/cp3,cp3,53853197,8413220,6510
4,1626163,Frank Kaminsky,https://twitter.com/FSKPart3,FSKPart3,https://instagram.com/fskpart3,fskpart3,135026973,211534,7093


In [110]:
players_socials_stats.to_csv('players_socials_stats.csv', index=False)

## Get instagram stats

In [53]:
from requests import get
from bs4 import BeautifulSoup
import requests
import re
URL = "https://www.instagram.com/{}/"
 
# parse function
def parse_data(s):
     
    # creating a dictionary
    data = {}
     
    # splitting the content
    # then taking the first part
    s = s.split("-")[0]
     
    # again splitting the content
    s = s.split(" ")
     
    # assigning the values
    data['Followers'] = s[0]
    data['Following'] = s[2]
    data['Posts'] = s[4]
     
    # returning the dictionary
    return data
 
# scrape function
def scrape_data(username):
     
    # getting the request from url
    r = requests.get(URL.format(username))
     
    # converting the text
    s = BeautifulSoup(r.text, "html.parser")
     
    # finding meta info
    meta = s.find("meta", property ="og:description")
     
    # calling parse method
    return parse_data(meta.attrs['content'])

In [54]:
data = scrape_data('fskpart3')
data

{'Followers': '184k', 'Following': '455', 'Posts': '245'}

In [55]:
players_all_socials = pd.read_csv('players_socials_stats.csv')
followers_column = []
post_column = []
for index, player in players_all_socials.iterrows():
    if pd.isna(player['insta_handle']) == False:
        try:
            info = scrape_data(player['insta_handle'])
            followers_column.append(info['Followers'])
            post_column.append(info['Posts'])
        except:
            print(player['name'])
    else:
        followers_column.append(0)
        post_column.append(0)
    time.sleep(30)

Jalen Smith


KeyboardInterrupt: 

In [15]:
from instascrape import *
google = Profile('https://instagram.com/fskpart3')
#google_post = Post('https://www.instagram.com/p/CG0UU3ylXnv/')
#google_hashtag = Hashtag('https://www.instagram.com/explore/tags/google/')

# Scrape their respective data 
google.scrape()
#google_post.scrape()
#google_hashtag.scrape()

print(google.followers)
#print(google_post['hashtags'])
#print(google_hashtag.amount_of_posts)

InstagramLoginRedirectError: Instagram is redirecting you to the login page instead of the page you are trying to scrape. This could be occuring because you made too many requests too quickly or are not logged into Instagram on your machine. Try passing a valid session ID to the scrape method as a cookie to bypass the login requirement

In [19]:
from selenium import webdriver
# Initialize the driver
driver = webdriver.Chrome()
# Open provided link in a browser window using the driver
driver.get("https://google.com")
time.sleep(5)
driver.quit

<bound method ChromiumDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="a4283210cef741369935727df70b8f65")>>

In [33]:
import instaloader
loader = instaloader.Instaloader()
 
# Login using the credentials
loader.login('r_phil', '5606rips')
 
# Use Profile class to access metadata of account
profile = instaloader.Profile.from_username(loader.context,
                                            'geeks_for_geeks')
profile

<Profile geeks_for_geeks (6898786619)>

In [35]:
followers = profile.get_followers()
followers

In [21]:
from instascrape import *
import pandas as pd
from selenium.webdriver import Chrome 
webdriver = Chrome()
session = '461379648%3AoYnYTUMdmjTYFe%3A26
sessionid = 'sessionid='+session
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": sessionid
}

players_all_socials = pd.read_csv('players_socials_stats.csv')
followers_column = []
post_column = []
for index, player in players_all_socials.iterrows():
    if pd.isna(player['insta_handle']) == False:
        try:
            info = Profile(player['insta_url'])
            info.scrape(headers=headers)
            followers_column.append(info.followers)
            post_column.append(info.posts)
        except:
            print(player['name'])
    else:
        followers_column.append(0)
        post_column.append(0)
    time.sleep(10)
        

JaVale McGee
Devin Booker


KeyboardInterrupt: 

## combine salary and social stats

In [198]:
salary = pd.read_csv('salary.csv')
players_socials_stats = pd.read_csv('players_socials_stats.csv')
salary_s = salary.groupby(['Player'])[['2021-22','2022-23','2023-24','2024-25','2025-26','2026-27','Guaranteed']].sum()
players_socials_stats

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count
0,201580,JaVale McGee,https://twitter.com/JaValeMcGee,JaValeMcGee,https://instagram.com/javalemcgee,javalemcgee,24838521,365149,12454
1,1626164,Devin Booker,https://twitter.com/DevinBook,DevinBook,https://instagram.com/dbook,dbook,237073728,1089895,9161
2,203901,Elfrid Payton,https://twitter.com/elfrid,elfrid,https://instagram.com/elfrid,elfrid,253527338,66291,10892
3,101108,Chris Paul,https://twitter.com/CP3,CP3,https://instagram.com/cp3,cp3,53853197,8413220,6510
4,1626163,Frank Kaminsky,https://twitter.com/FSKPart3,FSKPart3,https://instagram.com/fskpart3,fskpart3,135026973,211534,7093
...,...,...,...,...,...,...,...,...,...
497,1630243,Trevelin Queen,https://twitter.com/trev_up,trev_up,https://instagram.com/trevupnow_,trevupnow_,3335097551,3381,8718
498,1629726,Garrison Mathews,https://twitter.com/gmathews_24,gmathews_24,https://instagram.com/garrison_mathews24,garrison_mathews24,3103332430,7030,1640
499,1628464,Daniel Theis,https://twitter.com/dtheis10,dtheis10,https://instagram.com/dtheis10,dtheis10,4044682379,51870,1393
500,1630578,Alperen Sengun,https://twitter.com/alperennsengun,alperennsengun,https://instagram.com/alperen.sengun,alperen.sengun,1101806852034232320,51526,60


In [264]:
salary = pd.read_csv('hhsalary.csv')
players_socials_stats = pd.read_csv('players_socials_stats.csv')
players_socials_stats['name']=players_socials_stats['name'].replace({'\.': '', '\'':''}, regex=True)
players_socials_stats[players_socials_stats['name']=='PJ Tucker']

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count
25,200782,PJ Tucker,NaN,NaN,https://instagram.com/pjtucker,pjtucker,0,0,0


In [265]:
salary_socials = players_socials_stats.join(salary.set_index(['adj_name']), on='name')
#salary_socials['2021-22'] = salary_socials['2021-22'].fillna(53176)
#salary_socials.to_csv('salary_socials.csv', index=False)
salary_socials[salary_socials['name']=='PJ Tucker']

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count,Name,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27
25,200782,PJ Tucker,NaN,NaN,https://instagram.com/pjtucker,pjtucker,0,0,0,PJ Tucker,7000000,7350000,0,0,0,0


In [266]:
salary_socials[salary_socials['2021-22'].isna() == True]


,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count,Name,2021-22,2022-23,2023-24,2024-25,2025-26,2026-27


In [270]:
output = salary_socials[['playerid', 'name', 'twit_followers_count','2021-22']]
output['followers per dollar'] = salary_socials['twit_followers_count']/salary_socials['2021-22']
sorto = output.sort_values(by='followers per dollar', ascending=False)
sorto.to_csv('output.csv', index = False)

/var/folders/tl/ljh9dpdd2fnbgmc62qc2b49h0000gn/T/ipykernel_4279/3285655897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['followers per dollar'] = salary_socials['twit_followers_count']/salary_socials['2021-22']


# BUILD PLAYERS STATS FROM BB REFERENCE

## PULL FROM ALL PLAYERS STATS

### Download website info

In [8]:
import requests
import os
import shutil
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome(
    executable_path="/Users/HomeFolder/Documents/GitHub/NBAKlout/chromedriver"
    )
# install selenium chrome driver from https://chromedriver.chromium.org/downloads
# xattr -d com.apple.quarantine chromedriver

/var/folders/tl/ljh9dpdd2fnbgmc62qc2b49h0000gn/T/ipykernel_3590/4280033286.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [11]:
#download html from website
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_2022_{}.html"
stat_tables = ["totals","per_game","per_minute","per_poss","advanced","play-by-play","shooting","adj_shooting"]
for table in stat_tables:
    url = player_stats_url.format(table)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    
    with open("bbr/{}.html".format(table), "w+") as f:
        f.write(driver.page_source)

### build tables from html

In [192]:
import requests
import os
import shutil
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
stat_tables = ["totals","per_game","per_minute","per_poss","advanced","play-by-play","shooting","adj_shooting"]
dfs = {}
for table in stat_tables:
    with open("bbr/{}.html".format(table)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    while soup.find('tr', class_="thead") is not None:
        soup.find('tr', class_="thead").decompose()
    if table == "play-by-play":
        player_table = soup.find_all(id="pbp_stats")[0]
    elif table == "adj_shooting":
        player_table = soup.find_all(id="adj-shooting")[0]
    else:
        player_table = soup.find_all(id=table+"_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    
    if table in ["play-by-play", "adj_shooting", "shooting"]: 
        columns = []
        for col in list(player_df.columns):
            if col[0].find("Unnamed") == -1:
                 columns.append(col[0]+"_"+col[1])
            else:
                columns.append(col[1])
        player_df.columns = columns

    if table == 'totals':
        del player_df['Rk']
    elif table == 'adj_shooting':
        player_df.drop(['Rk','Pos','Age','Team'], axis=1, inplace=True)
    elif table == 'shooting':
        (player_df.drop(['Rk','Pos','Age','Tm','Unnamed: 9_level_1','Unnamed: 16_level_1',
                         'Unnamed: 23_level_1','Unnamed: 26_level_1','Unnamed: 29_level_1',
                         'Unnamed: 32_level_1'], axis=1, inplace=True))
    else:
        player_df.drop(['Rk','Pos','Age','Tm'], axis=1, inplace=True)
#    columns = []
#    for col in list(player_df.columns):
#        if table in ["play-by-play", "adj_shooting", "shooting"]: 
#            if col[0].find("Unnamed") == -1:
#                columns.append(table+"_"+col[0]+"_"+col[1])
#            else:
#                columns.append(table+"_"+col[1])
#        else:
#            columns.append(table+"_"+col)
#    player_df.columns = columns
    player_df.set_index('Player')
    dfs[table] = player_df

In [197]:
for df in dfs.keys():
    dfs[df].to_csv("{}.csv".format(df), index=False)

## Clean tables

### Deal with duplicates

#### Totals table - test

In [270]:
totals = dfs['totals']
totals_dup = totals[totals.duplicated(subset=['Player'], keep=False)]
totals_clean = totals.drop_duplicates(subset=['Player'], keep='first')

In [222]:
#supertotals = totals_dup.groupby(['Player', 'Age']).sum()
#supertotals.head().columns

Index(['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [223]:
#supertotals.loc[:,'FG%'] = supertotals['FG']/supertotals['FGA']
#supertotals.loc[:,'3P%'] = supertotals['3P']/supertotals['3PA']
#supertotals.loc[:,'2P%'] = supertotals['2P']/supertotals['2PA']
#supertotals.loc[:,'FT%'] = supertotals['FT']/supertotals['FTA']
#supertotals.loc[:,'eFG%'] = (supertotals['FG'] + (0.5*supertotals['3P']))/supertotals['FGA']

In [260]:
#total_enter = totals_dup.drop_duplicates(subset=['Player'], keep='last')[['Player','Pos','Tm']]
#total_enter = total_enter.set_index('Player').join(supertotals)
#total_enter = total_enter.reset_index()
#totals_clean = totals_clean.append(total_enter)


In [271]:
totals_clean.to_csv("clean_totals.csv",index=False)

#### All tables

In [313]:
import unicodedata
for df in dfs.keys():   
    dfs_clean = dfs[df].drop_duplicates(subset=['Player'], keep='first')
    dfs_clean.to_csv("clean_{}.csv".format(df), index=False)
 #   dfs_clean = pd.read_csv("clean_{}.csv".format(df))
 #   dfs_clean['Player'] = (dfs_clean['Player']).map(lambda x: unicodedata.normalize('NFKD', x)).str.encode('ascii', 'ignore').str.decode('utf-8')
#   dfs_clean.to_csv("clean_{}.csv".format(df), index=False)



# BUILD PLAYERS TABLE FROM NBA STATS

# BUILD STATS TABLE

## Build super table

In [329]:
import unicodedata
players_socials_stats = pd.read_csv("players_socials_stats.csv")
clean_totals = pd.read_csv("clean_totals.csv")
br_urls= pd.read_csv("players_br_urls.csv")
clean_totals['Player'] = (clean_totals['Player']).map(lambda x: unicodedata.normalize('NFKD', x)).str.encode('ascii', 'ignore').str.decode('utf-8')
stats = players_socials_stats.join(clean_totals.set_index('Player'), on='name')
stats.columns
#clean_totals.to_csv("text.csv")


Index(['playerid', 'name', 'twit_url', 'twit_handle', 'insta_url',
       'insta_handle', 'twit_id', 'twit_followers_count', 'twit_tweet_count',
       'tweet_retweet_count', 'tweet_reply_count', 'tweet_like_count',
       'tweet_quote_count', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [330]:
clean_shooting = pd.read_csv("clean_shooting.csv")
clean_shooting['Player'] = (clean_shooting['Player']).map(lambda x: unicodedata.normalize('NFKD', x)).str.encode('ascii', 'ignore').str.decode('utf-8')
clean_shooting = clean_shooting.drop(columns=['G', 'MP', 'FG%'])
clean_shooting = clean_shooting.fillna(0)
stats = stats.join(clean_shooting.set_index('Player'), on='name').convert_dtypes()
stats

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count,tweet_retweet_count,...,FG% by Distance_16-3P,FG% by Distance_3P,% of FG Ast'd_2P,% of FG Ast'd_3P,Dunks_%FGA,Dunks_#,Corner 3s_%3PA,Corner 3s_3P%,Heaves_Att.,Heaves_#
0,201580,JaVale McGee,https://twitter.com/JaValeMcGee,JaValeMcGee,https://instagram.com/javalemcgee,javalemcgee,24838521,365535,12457,26074,...,0.429,0.4,0.713,1.0,0.229,69,0.0,0.0,0,0
1,1626164,Devin Booker,https://twitter.com/DevinBook,DevinBook,https://instagram.com/dbook,dbook,237073728,1095730,9162,72373,...,0.472,0.373,0.398,0.664,0.013,11,0.134,0.378,0,0
2,203901,Elfrid Payton,https://twitter.com/elfrid,elfrid,https://instagram.com/elfrid,elfrid,253527338,66416,10894,4195,...,0.25,0.167,0.261,0.5,0.04,4,0.333,0.0,0,0
3,101108,Chris Paul,https://twitter.com/CP3,CP3,https://instagram.com/cp3,cp3,53853197,8418898,6514,14737,...,0.492,0.339,0.116,0.421,0.0,0,0.083,0.286,3,0
4,1626163,Frank Kaminsky,https://twitter.com/FSKPart3,FSKPart3,https://instagram.com/fskpart3,fskpart3,135026973,211419,7091,2808,...,0.0,0.333,0.71,1.0,0.076,4,0.067,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,1630243,Trevelin Queen,https://twitter.com/trev_up,trev_up,https://instagram.com/trevupnow_,trevupnow_,3335097551,3374,8722,309,...,0.0,0.364,0.6,0.5,0.034,1,0.136,0.333,0,0
498,1629726,Garrison Mathews,https://twitter.com/gmathews_24,gmathews_24,https://instagram.com/garrison_mathews24,garrison_mathews24,3103332430,7044,1641,10407,...,0.0,0.377,0.793,1.0,0.027,8,0.275,0.343,0,0
499,1628464,Daniel Theis,https://twitter.com/dtheis10,dtheis10,https://instagram.com/dtheis10,dtheis10,4044682379,53218,1396,2340,...,0.25,0.291,0.77,1.0,0.162,24,0.203,0.375,1,0
500,1630578,Alperen Sengun,https://twitter.com/alperennsengun,alperennsengun,https://instagram.com/alperen.sengun,alperen.sengun,1101806852034232320,53032,61,4531,...,0.0,0.259,0.386,0.933,0.092,23,0.034,0.5,6,0


In [331]:
stats.to_csv("stats.csv", index=False)


In [5]:
import pandas as pd
hhsocial=pd.read_csv("hhsocial.csv")
stats = pd.read_csv("stats.csv")
salary = hhsocial[['playerid','2021-22']]
salary = salary.rename(columns={'2021-22':'salary_2021_22'})
stats = stats.join(salary.set_index('playerid'), on='playerid')
stats.to_csv("stats.csv", index=False)

In [14]:
br_urls = pd.read_csv("players_br_urls.csv")
stats = pd.read_csv("stats.csv")
br_urls = br_urls.rename(columns={"name":"br_name"})
stats = stats.join(br_urls.set_index('playerid'), on='playerid')
stats.to_csv("stats.csv", index=False)

In [32]:
stats = pd.read_csv("stats.csv")
col = list(map(lambda x: x.lower(), list(stats.columns)))
col = list(map(lambda x: x.replace(" ","_"), col))
col = list(map(lambda x: x.replace("%","pct"), col))
col = list(map(lambda x: x.replace("-","to"), col))
col = list(map(lambda x: x.replace("#","num"), col))
col = list(map(lambda x: x.replace(".",""), col))
col = list(map(lambda x: x.replace("'",""), col))
stats.columns = col
stats = stats.convert_dtypes()
stats.to_csv("stats.csv", index=False)


In [8]:
list(stats.columns)

['playerid',
 'name',
 'twit_url',
 'twit_handle',
 'insta_url',
 'insta_handle',
 'twit_id',
 'twit_followers_count',
 'twit_tweet_count',
 'tweet_retweet_count',
 'tweet_reply_count',
 'tweet_like_count',
 'tweet_quote_count',
 'pos',
 'age',
 'tm',
 'g',
 'gs',
 'mp',
 'fg',
 'fga',
 'fgpct',
 '3p',
 '3pa',
 '3ppct',
 '2p',
 '2pa',
 '2ppct',
 'efgpct',
 'ft',
 'fta',
 'ftpct',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'dist',
 'pct_of_fga_by_distance_2p',
 'pct_of_fga_by_distance_0to3',
 'pct_of_fga_by_distance_3to10',
 'pct_of_fga_by_distance_10to16',
 'pct_of_fga_by_distance_16to3p',
 'pct_of_fga_by_distance_3p',
 'fgpct_by_distance_2p',
 'fgpct_by_distance_0to3',
 'fgpct_by_distance_3to10',
 'fgpct_by_distance_10to16',
 'fgpct_by_distance_16to3p',
 'fgpct_by_distance_3p',
 'pct_of_fg_astd_2p',
 'pct_of_fg_astd_3p',
 'dunks_pctfga',
 'dunks_num',
 'corner_3s_pct3pa',
 'corner_3s_3ppct',
 'heaves_att',
 'heaves_num',
 '2021to22',
 'salary_2021_22',
 'b

## Build Definitions table

In [4]:
defs = pd.DataFrame.from_dict({'Followers (Twitter)':'twit_followers_count',
    'Posts (Twitter)':'twit_tweet_count',
 'Retweets (Twitter)':'tweet_retweet_count',
 'Comments (Twitter)':'tweet_reply_count',
 'Likes (Twitter)':'tweet_like_count',
 'Quotes (Twitter)':'tweet_quote_count',
 'Likes Per Post (Twitter)':'twit_likes_per_post', 
 'Followers (Instagram)':'inst_followers',
 'Follows (Instagram)': 'inst_follows',
 'Posts (Instagram)': 'inst_posts',
 'Likes-Last 12 (Instagram)':'inst_likes_12',
 'Comments-Last 12 (Instagram)': 'inst_comments_12',
 'Likes Per Week (Instagram)': 'inst_comments_per_week',
 'Comments Per Week (Instagram)': 'inst_comments_per_week',                             
 'GP (Total Games Played)':'g',
  'GS (Total Games Started)':'gs',
  'MP (Total Minutes Played)':'mp',
  'FG (Total Field Goals Made)':'fg',
 'FGA (Total Field Goals Attempted)':'fga',
  'FG% (Field Goal %)':'fgpct',
 '3P (Total 3s Made)':'threep',
 '3PA (Total 3s Attempted)':'threepa',
 '3P% (3s % Made)': 'threeppct',
  '2P (Total 2s Made)':'twop',
  '2PA (Total 2s Attempted)':'twopa',
  '2P% (2s % Made)':'twoppct',
 'eFG% (Efficient FG%)':'efgpct',
  'FT (Total Free Throws Made)':'ft',
 'FTA (Total Free Throws Attempted)':'fta',
  'FT% (Free Thows % Made)':'ftpct',
 'ORB (Total Off Rebounds)':'orb',
 'DRB (Total Def Rebounds)':'drb',
 'TRB (Total Rebounds)':'trb',
 'AST (Total Assists)':'ast',
 'STL (Total Steals)':'stl',
 'BLK (Total Blocks)':'blk',
 'TOV (Total Turnovers)':'tov',
 'PF (Total Fouls)':'pf',
 'PTS (Total Points)':'pts',
 'Dist (Avg Shot Distance)':'dist',
 '% of FGA (2s)':'pct_of_fga_by_distance_2p',
 '% of FGA (0-3ft)':'pct_of_fga_by_distance_0to3',
 '% of FGA (3-10ft)': 'pct_of_fga_by_distance_3to10',
 '% of FGA (10-16ft)':'fgpct_by_distance_10to16',
 '% of FGA (16ft-3p)':'fgpct_by_distance_16to3p',
 '% of FGA (3p)':'fgpct_by_distance_3p',
 'FG% (0-3ft)':'fgpct_by_distance_0to3',
 'FG% (3-10ft)':'fgpct_by_distance_3to10',
 'FG% (10-16ft)':'fgpct_by_distance_10to16',
 'FG% (16ft-3p)':'fgpct_by_distance_16to3p',
 '% of FG (2s assisted)':"pct_of_fg_astd_2pP",
 '% of FG (3s assisted)':"pct_of_fg_astd_3p",
 '% of FGA (Dunks)':'dunks_pctfga',
 'Total Dunks':'dunks_num',
 '% of 3PA (Corner 3)':'corner_3s_pct3pa',
 '3P% (Corner 3)':'corner_3s_3ppct',
 'Heaves Attempted':'heaves_att',
 'Heaves Made':'heaves_num',
 'Dollars: Salary': 'salary_2021_22'                           
 }, orient='index').reset_index().rename({'index': 'desc', '0': 'column'}, axis=1)
defs.to_csv("defs.csv", index=False)

## Upload to sql

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import inspect
import psycopg2
import pandas as pd
instance = 'aaogmh364evezw'
dbname = 'Players'
username = 'nbakloutadmin'
passwd = 'nbaklout_admin'
hostname = 'aaogmh364evezw.clkrvvajsy5p.us-east-1.rds.amazonaws.com'
portnum = '5432'

engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(username,passwd,hostname,portnum,dbname))
session = scoped_session(sessionmaker(bind=engine))
inspector = inspect(engine)

conn = engine.connect()



### upload stats

In [2]:
stats = pd.read_csv("stats.csv")
#stats['twit_likes_per_post'] = stats['tweet_like_count']/stats['twit_tweet_count']
stats.to_sql('stats', con=engine, if_exists='replace')
#stats.to_csv("stats.csv")
#stats.columns


In [3]:
test = pd.read_sql("select threep from stats;",conn)
test

,threep
0,2.0
1,125.0
2,2.0
3,57.0
4,5.0
...,...
497,8.0
498,92.0
499,23.0
500,15.0


### upload defs

In [5]:
defs.to_sql('defs', con=engine, if_exists='replace', index=False)
test = pd.read_sql("SELECT  * from defs",conn)
test

,desc,0
0,Followers (Twitter),twit_followers_count
1,Posts (Twitter),twit_tweet_count
2,Retweets (Twitter),tweet_retweet_count
3,Comments (Twitter),tweet_reply_count
4,Likes (Twitter),tweet_like_count
5,Quotes (Twitter),tweet_quote_count
6,Likes Per Post (Twitter),twit_likes_per_post
7,Followers (Instagram),inst_followers
8,Follows (Instagram),inst_follows
9,Posts (Instagram),inst_posts


In [6]:
conn.close()
session.close()
engine.dispose()

# INSTAGRAM SCRAPE

## using selenium

In [60]:
import requests
import os
import shutil
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome(
    executable_path="/Users/HomeFolder/Documents/GitHub/NBAKlout/chromedriver"
    )
time.sleep(3)

/var/folders/tl/ljh9dpdd2fnbgmc62qc2b49h0000gn/T/ipykernel_67906/140045413.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [61]:
url = "https://www.instagram.com/"
driver.get(url)
time.sleep(3)
user = driver.find_element_by_xpath("//input[@name='username']")
user.send_keys(instauser)
passw = driver.find_element_by_xpath("//input[@name='password']")
passw.send_keys(instapass)

/var/folders/tl/ljh9dpdd2fnbgmc62qc2b49h0000gn/T/ipykernel_67906/3212012759.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  user = driver.find_element_by_xpath("//input[@name='username']")
/var/folders/tl/ljh9dpdd2fnbgmc62qc2b49h0000gn/T/ipykernel_67906/3212012759.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  passw = driver.find_element_by_xpath("//input[@name='password']")


In [62]:
from selenium.webdriver.common.by import By
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
login_button.click()
time.sleep(3)

In [63]:
next_button = driver.find_element(By.XPATH, "//button[text()='Not Now']")
next_button.click()
time.sleep(2)

In [64]:
next_button2 = driver.find_element(By.XPATH, "//button[text()='Not Now']")
next_button2.click()
time.sleep(2)

In [19]:
url2 = "https://www.instagram.com/r_phil/channel/?__a=1"
driver.get(url2)

In [22]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [27]:
import json
site_json=json.loads(soup.body.pre.text)

In [32]:
inst_id = site_json['graphql']['user']['id']
inst_username = site_json['graphql']['user']['username']
inst_followers = site_json['graphql']['user']['edge_followed_by']['count']
inst_follows = site_json['graphql']['user']['edge_follow']['count']
inst_posts = site_json['graphql']['user']['edge_owner_to_timeline_media']['count']
nextpage = site_json['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
endcursor = site_json['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']

In [71]:
import pandas as pd
import numpy as np
import requests
import time
from os.path import exists as file_exists
social = pd.read_csv("players_socials_stats.csv")
result = []
for i, player in social.iterrows():
    if file_exists("json/{}.html".format(player['name'])):
        continue
    if pd.isna(player['insta_handle']):
        continue
    try:
        url = f"https://www.instagram.com/{player['insta_handle']}/channel/?__a=1"
        driver.get(url)   
        with open("json/{}.html".format(player['name']), "w+") as f:
            f.write(driver.page_source)
        time.sleep(60)
    except Exception as e:
        print(e, player['name'])
        time.sleep(60)  

## build instagram stats table

In [23]:
from bs4 import BeautifulSoup
import json
from datetime import datetime
with open("json/{}.html".format("Anthony Gill")) as f:
    page = f.read()

soup = BeautifulSoup(page, 'html.parser')
site_json=json.loads(soup.body.pre.text)
edges = site_json['graphql']['user']['edge_owner_to_timeline_media']['edges']
counts=[0,0,0]
dates = []
today = datetime(2022, 2, 16)
start = datetime(2021, 10, 19)
for edge in edges:
    date = datetime.utcfromtimestamp(edge['node']['taken_at_timestamp'])
    if date >= start:
        counts[0] += edge['node']['edge_liked_by']['count']
        counts[1] += edge['node']['edge_media_to_comment']['count']
        counts[2] += 1
        dates.append(date)
counts, (today - min(dates)).days/7
 


([1979, 49, 2], 11.571428571428571)

In [10]:
from bs4 import BeautifulSoup
import json
from os.path import exists as file_exists
from datetime import datetime
import pandas as pd
import numpy as np
social = pd.read_csv("players_socials_stats.csv")
insta_stats = []
for i, player in social.iterrows():
    if file_exists("json/{}.html".format(player['name'])):
        with open("json/{}.html".format(player['name'])) as f:
            page = f.read()

        soup = BeautifulSoup(page, 'html.parser')
        inst_likes_12 = 0
        inst_comments_12 = 0
        inst_likes_per_week = 0
        inst_comments_per_week = 0
        dates = []
        today = datetime(2022, 2, 16)
        start = datetime(2021, 10, 19)
        try:
            site_json=json.loads(soup.body.pre.text)
            inst_id = site_json['graphql']['user']['id']
            #inst_username = site_json['graphql']['user']['username']
            inst_followers = site_json['graphql']['user']['edge_followed_by']['count']
            inst_follows = site_json['graphql']['user']['edge_follow']['count']
            inst_posts = site_json['graphql']['user']['edge_owner_to_timeline_media']['count']
            
            edges = site_json['graphql']['user']['edge_owner_to_timeline_media']['edges']
            for edge in edges:
                date = datetime.utcfromtimestamp(edge['node']['taken_at_timestamp'])
                if date >= start:
                    inst_likes_12 += edge['node']['edge_liked_by']['count']
                    inst_comments_12 += edge['node']['edge_media_to_comment']['count']
                    dates.append(date)
            if len(dates) > 0:
                weeks = (today - min(dates)).days/7
                inst_likes_per_week = inst_likes_12/weeks
                inst_comments_per_week = inst_comments_12/weeks
            (insta_stats.append((player['playerid'], int(inst_id), inst_followers, inst_follows,
                                 inst_posts, inst_likes_12,inst_comments_12,inst_likes_per_week,
                                 inst_comments_per_week)))
        except Exception as e:
            print(e, player['name'])
       # nextpage = site_json['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        #endcursor = site_json['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
    else:
        insta_stats.append((player['playerid'],0,0,0,0,0,0,0,0))
insta_out = (pd.DataFrame.from_records(insta_stats, columns = ['playerid', 'inst_id', 'inst_followers',
                                                               'inst_follows', 'inst_posts', 'inst_likes_12',
                                                               'inst_comments_12','inst_likes_per_week',
                                                               'inst_comments_per_week']))
insta_out.to_csv("insta_stats.csv", index=False)   

## add to stats table

In [11]:
stats = pd.read_csv("stats.csv")
insta = pd.read_csv("insta_stats.csv")
stats = (stats.drop(columns=['inst_id', 'inst_followers','inst_follows', 'inst_posts', 'inst_likes_12',
                            'inst_comments_12','inst_likes_per_week','inst_comments_per_week']))
stats = stats.join(insta.set_index('playerid'), on='playerid')
#stats = stats.drop(columns=['Unnamed: 0'])
stats.to_csv("stats.csv", index=False)
stats.columns

Index(['playerid', 'name', 'twit_url', 'twit_handle', 'insta_url',
       'insta_handle', 'twit_id', 'twit_followers_count', 'twit_tweet_count',
       'tweet_retweet_count', 'tweet_reply_count', 'tweet_like_count',
       'tweet_quote_count', 'pos', 'age', 'tm', 'g', 'gs', 'mp', 'fg', 'fga',
       'fgpct', '3p', '3pa', '3ppct', '2p', '2pa', '2ppct', 'efgpct', 'ft',
       'fta', 'ftpct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'dist', 'pct_of_fga_by_distance_2p',
       'pct_of_fga_by_distance_0to3', 'pct_of_fga_by_distance_3to10',
       'pct_of_fga_by_distance_10to16', 'pct_of_fga_by_distance_16to3p',
       'pct_of_fga_by_distance_3p', 'fgpct_by_distance_2p',
       'fgpct_by_distance_0to3', 'fgpct_by_distance_3to10',
       'fgpct_by_distance_10to16', 'fgpct_by_distance_16to3p',
       'fgpct_by_distance_3p', 'pct_of_fg_astd_2p', 'pct_of_fg_astd_3p',
       'dunks_pctfga', 'dunks_num', 'corner_3s_pct3pa', 'corner_3s_3ppct',
       'heaves_att', 'heave

# DEMO

## refresh twitter info

In [19]:
import tweepy
import pandas as pd
import numpy as np
import time

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAC%2FjXwEAAAAAUnmfELGGAifch1OktXC%2ByLE0DFs%3D3HaReF7sd1RWcZDuWVDLrZ5I0IMKp66apKOY4cbg1z1p0Y9NEU'
client = tweepy.Client(bearer_token)

#create blocks of 100
players_socials=pd.read_csv('players_socials_stats.csv')

In [20]:
players_socials[players_socials['playerid'] == 1629645]

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count
488,1629645,Kevin Porter Jr.,https://twitter.com/kevinporterjr,Kevinporterjr,https://www.instagram.com/kevinporterjr/,kevinporterjr,0,0,0


In [21]:
twit_blocks=[]
block = []
players_socials_stats = players_socials
for index, player in players_socials_stats.iterrows():
    if pd.isna(player['twit_handle']) == False :
        block.append(player['twit_handle'])
        if len(block) > 99:
            twit_blocks.append(block)
            block = []
if len(block) > 0:
    twit_blocks.append(block)

In [22]:
twitter_info = []
for block in twit_blocks:
    info = client.get_users(usernames=block, user_fields=["public_metrics"]).data
    twitter_info.extend(info)
    time.sleep(1)
#change twitter info into dictionary
twitter_info_dict = {}
for info in twitter_info:
    twitter_info_dict[info.username] = {'twit_id' : int(info.id),
                                        'twit_followers_count' : int(info.public_metrics['followers_count']),
                                        'twit_tweet_count' : int(info.public_metrics['tweet_count'])
                                       }
#add info
import numpy as np
id_column = []
followers_column = []
tweet_column = []

In [23]:
for index, player in players_socials_stats.iterrows():
    if pd.isna(player['twit_handle']) == False:
        if player['twit_handle'] in twitter_info_dict:
            info = twitter_info_dict[player['twit_handle']]
            id_column.append(int(info['twit_id']))
            followers_column.append(int(info['twit_followers_count']))
            tweet_column.append(int(info['twit_tweet_count']))
        ##clear incorrect socials
        else:
            players_socials['twit_url'] = players_socials['twit_url'].replace(player['twit_url'],np.nan)
            players_socials['twit_handle'] = players_socials['twit_handle'].replace(player['twit_handle'],np.nan)
            id_column.append(0)
            followers_column.append(0)
            tweet_column.append(0)
    else:
        id_column.append(0)
        followers_column.append(0)
        tweet_column.append(0)
players_socials_stats['twit_id'] = id_column
players_socials_stats['twit_followers_count'] = followers_column
players_socials_stats['twit_tweet_count'] = tweet_column

In [24]:
players_socials_stats[players_socials_stats['playerid'] == 1629645]

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count
488,1629645,Kevin Porter Jr.,https://twitter.com/kevinporterjr,Kevinporterjr,https://www.instagram.com/kevinporterjr/,kevinporterjr,2413645688,68797,1656


In [25]:
players_socials_stats.to_csv("players_socials_stats.csv", index=False)

## get tweets info

### test retrieving tweets

In [134]:
import tweepy
import pandas as pd
import numpy as np
import time

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAC%2FjXwEAAAAAUnmfELGGAifch1OktXC%2ByLE0DFs%3D3HaReF7sd1RWcZDuWVDLrZ5I0IMKp66apKOY4cbg1z1p0Y9NEU'
client = tweepy.Client(bearer_token)
playerid = 2413645688

start_time = "2021-10-19T00:00:00Z"
thetweets = tweepy.Paginator(client.get_users_tweets, playerid, start_time=start_time, tweet_fields=["created_at", "text", "public_metrics"],max_results=100).flatten(limit=3200)

In [ ]:
for tweet in tweepy.Paginator(client.search_recent_tweets, "Tweepy",
                              max_results=100).flatten(limit=3200):
    print(tweet.id)

In [109]:
t = client.get_users_tweets( player, start_time=start_time, tweet_fields=["created_at", "text", "public_metrics"],max_results=100)

In [135]:
#len(t.data),t.data[1].public_metrics, t.data[1].created_at, t.data[-1].created_at

r=[]
for tweet in thetweets:
    r.append((tweet.id, tweet.text, tweet.data['created_at'], tweet.data['public_metrics']))

In [141]:
r[0]

        
    

(1492304918660763650,
 '@NBA2K Ya’ll kno my Handle better than this.. I can’t even tween hesi 🗿',
 '2022-02-12T01:09:52.000Z',
 {'retweet_count': 119,
  'reply_count': 59,
  'like_count': 1597,
  'quote_count': 21})

In [140]:
t[0][0]
  

<Tweet id=1492304918660763650 text=@NBA2K Ya’ll kno my Handle better than this.. I can’t even tween hesi 🗿>

In [128]:
tel = []
for i in t:
    
tel

AttributeError: 'list' object has no attribute 'id'

### get tweets from dataframe

In [56]:
players_socials_stats.head(2)

,playerid,name,twit_url,twit_handle,insta_url,insta_handle,twit_id,twit_followers_count,twit_tweet_count
0,201580,JaVale McGee,https://twitter.com/JaValeMcGee,JaValeMcGee,https://instagram.com/javalemcgee,javalemcgee,24838521,365535,12457
1,1626164,Devin Booker,https://twitter.com/DevinBook,DevinBook,https://instagram.com/dbook,dbook,237073728,1095730,9162


In [73]:
tweetdata = players_socials_stats[['playerid','twit_id']]


In [143]:
import tweepy
import pandas as pd
import numpy as np
import time
import csv

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAC%2FjXwEAAAAAUnmfELGGAifch1OktXC%2ByLE0DFs%3D3HaReF7sd1RWcZDuWVDLrZ5I0IMKp66apKOY4cbg1z1p0Y9NEU'
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)
start_time = "2021-10-19T00:00:00Z"

tweetdata = players_socials_stats[['playerid','twit_id']].copy()
alldata = []
for index, player in tweetdata.iterrows():
    if player['twit_id'] != 0:
        manytweets = tweepy.Paginator(client.get_users_tweets, player['twit_id'], start_time=start_time, tweet_fields=["created_at", "text", "public_metrics"],max_results=100).flatten(limit=3200)
        for tweet in manytweets:
            (alldata.append((player['playerid'],player['twit_id'],tweet.id,
                               tweet.text, tweet.data['created_at'], tweet.data['public_metrics']['retweet_count'],
                               tweet.data['public_metrics']['reply_count'],tweet.data['public_metrics']['like_count'],
                               tweet.data['public_metrics']['quote_count']))) 
    time.sleep(2)

alldatadf = (pd.DataFrame.from_records(alldata, columns =['playerid', 'twit_id', 'tweet_id', 'tweet_text', 
                                                   'tweet_created_at', 'tweet_retweet_count','tweet_reply_count',
                                                  'tweet_like_count','tweet_quote_count',]))
alldatadf.to_csv("alldata.csv", index=False)

In [146]:
alldatadf.shape

(22268, 9)

In [151]:
tweetsums = alldatadf[['playerid', 'tweet_retweet_count','tweet_reply_count','tweet_like_count','tweet_quote_count']].groupby('playerid').sum()
tweetsums.to_csv("tweetsums.csv", index=False)

In [152]:
players_socials_stats = pd.read_csv("players_socials_stats.csv")

In [189]:
out = players_socials_stats.set_index('playerid').join(tweetsums).copy()
out['tweet_retweet_count'] = out['tweet_retweet_count'].fillna(0)
out['tweet_reply_count'] = out['tweet_reply_count'].fillna(0)
out['tweet_like_count'] = out['tweet_like_count'].fillna(0)
out['tweet_quote_count'] = out['tweet_quote_count'].fillna(0)
out = out.reset_index().convert_dtypes()
out.to_csv("players_socials_stats.csv", index=False)
#out.head()



## download Player profile pics

In [130]:
import requests
import os
import shutil
from bs4 import BeautifulSoup
import pandas as pd
import time
import shutil

df = pd.read_csv("players_br_urls.csv")
for player in df.iterrows():
    try:
        response = requests.get(player[1]['br_url'])
        soup = BeautifulSoup(response.text, "html.parser")
        url = soup.find('img',{"itemscope": "image"})["src"]
    except:
        print(player[1]['name'])
    time.sleep(2)
    r = requests.get(url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open("imgs/{}.jpg".format(player[1]['name']),'wb') as f:
            shutil.copyfileobj(r.raw, f)
    else:
        print(player[1]['name'])
    time.sleep(2)

Malcolm Hill


In [109]:
url

'https://www.basketball-reference.com/leagues/NBA_2022_adj_shooting.html'

In [128]:
response = requests.get("https://www.basketball-reference.com/players/h/hillma01.html")
soup = BeautifulSoup(response.text, "html.parser")
try :
    url = soup.find('img',{"itemscope": "image"})["src"]
    time.sleep(2)
    r = requests.get(url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open("imgs/{}.jpg".format("hmmm"),'wb') as f:
            shutil.copyfileobj(r.raw, f)
    time.sleep(2)
except:
    print("test")

test


## build demo table

In [147]:
hhsocial=pd.read_csv("hhsocial.csv")
br = pd.read_csv("players_br_urls.csv")
br.rename(columns={"name":"br_name"}, inplace=True)
demo = hhsocial.join(br.set_index('playerid'), on='playerid')

In [169]:
demo['name'].to_list()

['Carmelo Anthony',
 'Dwight Howard',
 'LeBron James',
 'DeMarcus Cousins',
 'Andre Iguodala',
 'Kevin Durant',
 'Andre Drummond',
 'Stephen Curry',
 'Lance Stephenson',
 'LaMarcus Aldridge',
 'Chris Paul',
 'Enes Freedom',
 'Rajon Rondo',
 'Rudy Gay',
 'Cassius Stanley',
 'Derrick Rose',
 'Theo Pinson',
 'James Harden',
 'Ricky Rubio',
 'Ja Morant',
 'Sharife Cooper',
 'Trae Young',
 'Russell Westbrook',
 'Udonis Haslem',
 'Luka Doncic',
 'Blake Griffin',
 'Victor Oladipo',
 'Wesley Matthews',
 'Kyrie Irving',
 'Facundo Campazzo',
 'Austin Rivers',
 'LaMelo Ball',
 'DeAndre Jordan',
 'Yuta Watanabe',
 'Kyle Guy',
 'Dennis Smith Jr',
 'Kevin Love',
 'Frank Kaminsky',
 'Serge Ibaka',
 'Cassius Winston',
 'Wenyen Gabriel',
 'Kyle Kuzma',
 'Nicolas Batum',
 'Malik Monk',
 'Tyler Herro',
 'Gary Payton II',
 'Damian Lillard',
 'Hassan Whiteside',
 'JaVale McGee',
 'Neemias Queta',
 'Patty Mills',
 'Lonzo Ball',
 'Admiral Schofield',
 'Tristan Thompson',
 'Draymond Green',
 'Taj Gibson',
 'A

In [156]:
demo.to_csv("demo.csv", index=False)

## upload to sql

In [157]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import inspect
import psycopg2
import pandas as pd

engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(username,passwd,hostname,portnum,dbname))
session = scoped_session(sessionmaker(bind=engine))
inspector = inspect(engine)

In [159]:
demo.to_sql('demo', con=engine, if_exists='replace')

In [164]:
engine.execute("SELECT * FROM demo").fetchmany(5)

[(0, 2546, 'Carmelo Anthony', 9266281, 2641691, 3.507708130890403, 'Carmelo Anthony', 'https://www.basketball-reference.com/players/a/anthoca01.html'),
 (1, 2730, 'Dwight Howard', 6383134, 2641691, 2.4163060706191604, 'Dwight Howard', 'https://www.basketball-reference.com/players/h/howardw01.html'),
 (2, 2544, 'LeBron James', 50745912, 41180544, 1.232278815937934, 'LeBron James', 'https://www.basketball-reference.com/players/j/jamesle01.html'),
 (3, 202326, 'DeMarcus Cousins', 1250353, 1203925, 1.0385638640280748, 'DeMarcus Cousins', 'https://www.basketball-reference.com/players/c/couside01.html'),
 (4, 2738, 'Andre Iguodala', 1326440, 2641691, 0.5021177722905518, 'Andre Iguodala', 'https://www.basketball-reference.com/players/i/iguodan01.html')]

In [165]:
conn = engine.connect()
demo1 = pd.read_sql("SELECT  * from demo",conn)
demo1

,index,playerid,name,twit_followers_count,2021-22,followers per dollar,br_name,br_url
0,0,2546,Carmelo Anthony,9266281,2641691,3.507708,Carmelo Anthony,https://www.basketball-reference.com/players/a...
1,1,2730,Dwight Howard,6383134,2641691,2.416306,Dwight Howard,https://www.basketball-reference.com/players/h...
2,2,2544,LeBron James,50745912,41180544,1.232279,LeBron James,https://www.basketball-reference.com/players/j...
3,3,202326,DeMarcus Cousins,1250353,1203925,1.038564,DeMarcus Cousins,https://www.basketball-reference.com/players/c...
4,4,2738,Andre Iguodala,1326440,2641691,0.502118,Andre Iguodala,https://www.basketball-reference.com/players/i...
...,...,...,...,...,...,...,...,...
497,497,1630264,Anthony Gill,0,1517981,0.000000,Anthony Gill,https://www.basketball-reference.com/players/g...
498,498,1629008,Michael Porter Jr,0,5258735,0.000000,Michael Porter Jr.,https://www.basketball-reference.com/players/p...
499,499,1630559,Austin Reaves,0,925258,0.000000,Austin Reaves,https://www.basketball-reference.com/players/r...
500,500,202722,Davis Bertans,0,16000000,0.000000,Davis Bertans,https://www.basketball-reference.com/players/b...


In [167]:
query = 'SELECT * FROM information_schema.tables;'
engine.execute(query).fetchall()

[('Players', 'public', 'Salary', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('Players', 'pg_catalog', 'pg_type', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('Players', 'public', 'demo', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('Players', 'pg_catalog', 'pg_foreign_table', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('Players', 'pg_catalog', 'pg_roles', 'VIEW', None, None, None, None, None, 'NO', 'NO', None),
 ('Players', 'pg_catalog', 'pg_settings', 'VIEW', None, None, None, None, None, 'NO', 'NO', None),
 ('Players', 'pg_catalog', 'pg_prepared_statements', 'VIEW', None, None, None, None, None, 'NO', 'NO', None),
 ('Players', 'pg_catalog', 'pg_stat_progress_vacuum', 'VIEW', None, None, None, None, None, 'NO', 'NO', None),
 ('Players', 'pg_catalog', 'pg_subscription', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None),
 ('Players', 'pg_catalog', 'pg_attribute', 'BASE TABLE', None, None, 